<a href="https://colab.research.google.com/github/elangbijak4/Image-Video-AI/blob/main/Rev2_GPU_Demonstrasi_DETR_Image_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.cuda.is_available())  # Harus mencetak True jika GPU tersedia
print(torch.cuda.get_device_name(0))  # Menampilkan nama GPU

True
Tesla T4


In [ ]:
# Instalasi pustaka yang diperlukan
!pip install transformers torch torchvision opencv-python-headless

In [ ]:
!pip install timm

In [4]:
import timm
#import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [5]:
# Import pustaka yang diperlukan
#import torch
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import requests
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [6]:
import cv2
import numpy as np

In [7]:
# Periksa apakah GPU tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
# Load model DETR dan prosesor gambar ke GPU
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Fungsi untuk menambahkan kotak deteksi ke dalam frame
def draw_boxes(frame, prob, boxes):
    colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k']
    for i, (p, (xmin, ymin, xmax, ymax)) in enumerate(zip(prob, boxes.tolist())):
        # Convert koordinat ke integer
        xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])
        # Gambar kotak deteksi
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        if isinstance(p, torch.Tensor):
            if p.numel() > 1:  # Check if p has more than one element
                cl = torch.argmax(p)
                text = f'{model.config.id2label[cl.item()]}: {p[cl].item():0.2f}'
            else:  # Handle the case where p is a scalar
                text = f'{model.config.id2label[0]}: {p.item():0.2f}'
        else:
            cl = p.index(max(p))  # Untuk kasus list
            text = f'{model.config.id2label[cl]}: {max(p):0.2f}'
        cv2.putText(frame, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    return frame

In [10]:
# URL video contoh
video_url = "https://www.learningcontainer.com/wp-content/uploads/2020/05/sample-mp4-file.mp4"
video_path = "sample_video.mp4"

# Download video
!wget {video_url} -O {video_path}

# Baca video menggunakan OpenCV
cap = cv2.VideoCapture(video_path)

# Mendapatkan lebar, tinggi, dan FPS video asli
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Tentukan codec dan buat VideoWriter untuk menyimpan hasil
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (width, height))

--2024-06-21 16:49:29--  https://www.learningcontainer.com/wp-content/uploads/2020/05/sample-mp4-file.mp4
Resolving www.learningcontainer.com (www.learningcontainer.com)... 188.114.96.0, 188.114.97.0, 2a06:98c1:3120::, ...
Connecting to www.learningcontainer.com (www.learningcontainer.com)|188.114.96.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10546620 (10M) [video/mp4]
Saving to: ‘sample_video.mp4’

sample_video.mp4    100%[===================>]  10.06M  --.-KB/s    in 0.05s   

2024-06-21 16:49:29 (191 MB/s) - ‘sample_video.mp4’ saved [10546620/10546620]



In [11]:
# Proses setiap frame dalam video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame ke format yang bisa diproses oleh PIL
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame_rgb)

    # Proses gambar
    inputs = processor(images=image, return_tensors="pt").to(device)
    outputs = model(**inputs)

    # Peroleh prediksi
    target_sizes = torch.tensor([image.size[::-1]]).to(device)
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]

    # Filter prediksi dengan probabilitas tinggi
    prob_threshold = 0.7
    scores = results["scores"]
    boxes = results["boxes"]

    # Pastikan ada deteksi yang valid
    if scores.numel() > 0 and boxes.numel() > 0:
        keep = scores > prob_threshold
        if keep.sum().item() > 0:  # Pastikan ada deteksi yang tersisa setelah filter
            # Filter tensor berdasarkan indeks yang dipilih
            filtered_scores = scores[keep]
            filtered_boxes = boxes[keep]
            # Gambar kotak deteksi pada frame
            frame = draw_boxes(frame, filtered_scores, filtered_boxes)

    # Simpan frame hasil ke video output
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print("Video hasil telah disimpan sebagai 'output_video.mp4'")

Video hasil telah disimpan sebagai 'output_video.mp4'
